#  Maintenance des tables Delta Lake
- Analyse des logs Delta 
- Historique de la table
- Vacuum
- Optimisations


In [2]:
df = spark.sql("SELECT * FROM lakehouse_gold.sales LIMIT 10")
display(df)

StatementMeta(, ae035dc8-4c2d-40a2-9f5f-4213487e99e5, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c07ed5ae-0290-4d6d-b4e2-7eb3682f3e26)

# Analyse des logs Delta

In [3]:
files_list = mssparkutils.fs.ls("abfss://wwi_03@onelake.dfs.fabric.microsoft.com/lakehouse_gold.Lakehouse/Tables/sales")
display(files_list)

StatementMeta(, ae035dc8-4c2d-40a2-9f5f-4213487e99e5, 5, Finished, Available, Finished)

[FileInfo(path=abfss://wwi_03@onelake.dfs.fabric.microsoft.com/lakehouse_gold.Lakehouse/Tables/sales/Year=2000, name=Year=2000, size=0),
 FileInfo(path=abfss://wwi_03@onelake.dfs.fabric.microsoft.com/lakehouse_gold.Lakehouse/Tables/sales/_delta_log, name=_delta_log, size=0),
 FileInfo(path=abfss://wwi_03@onelake.dfs.fabric.microsoft.com/lakehouse_gold.Lakehouse/Tables/sales/_temporary, name=_temporary, size=0)]

In [4]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,LongType
data=[]
for i in files_list:
    d={}   
    d["path"]=i.path
    d["name"]=i.name
    d["size"]=i.size
    d["modifyTime"]=i.modifyTime
    data.append(d)
schema = StructType([
    StructField("path", StringType(), True),
    StructField("name", StringType(), True),
    StructField("size", LongType(), True),
    StructField("modifyTime", LongType(), True),
])
df = spark.createDataFrame(data, schema=schema)
display(df)

StatementMeta(, ae035dc8-4c2d-40a2-9f5f-4213487e99e5, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0905c503-a52c-4bb3-8e78-d2b615770ca6)

# Historique de la table

In [5]:
%%sql
-- Table history
DESCRIBE HISTORY sales

StatementMeta(, ae035dc8-4c2d-40a2-9f5f-4213487e99e5, 7, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 15 fields>

# Vacuum
Vacuum supprime les anciens fichiers qui ne sont plus référencés par un journal de table Delta. 
Le seuil de rétention des fichiers par défaut est de sept jours. 
Lorsque vous exécutez Vacuum avec la période de rétention par défaut, les fichiers plus anciens que le seuil de rétention sont supprimés. 

In [6]:
%%sql
VACUUM sales

StatementMeta(, ae035dc8-4c2d-40a2-9f5f-4213487e99e5, 8, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [ ]:
%%sql
--https://docs.delta.io/latest/delta-batch.html#data-retention
-- delta.logRetentionDuration: Controls how long the history for a table is kept. The default is interval 30 days. 
-- delta.deletedFileRetentionDuration: Controls how long ago a file must have been deleted before being a candidate for VACUUM. The default is interval 7 days. 
--spark.sql("ALTER TABLE delta sales SET TBLPROPERTIES ('delta.logRetentionDuration'='2 days')")
--spark.sql("ALTER TABLE delta sales SET TBLPROPERTIES ('delta.deletedFileRetentionDuration'='30 days')")

# V-Order
Le tri V-Order augmente de 15 % le temps d’écriture moyen mais permet jusqu’à 50 % de compression en plus et des temps d’accès aux données optimisés.

In [7]:
%%sql
-- You can check if v-order is enabled for your spark session 
SET spark.sql.parquet.vorder.enabled 

StatementMeta(, ae035dc8-4c2d-40a2-9f5f-4213487e99e5, 9, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 2 fields>

# Optimisations
Afin de maintenir les tables dans un état optimal pour de meilleures performances, la commande OPTIMIZE fusionne les modifications dans des fichiers Parquet plus grands et consolidés. 
La commande OPTIMIZE peut être combinée avec la commande Z-ordering. 

In [8]:
%%sql
OPTIMIZE sales ZORDER BY CityKey VORDER; 

StatementMeta(, ae035dc8-4c2d-40a2-9f5f-4213487e99e5, 10, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 2 fields>